# In The Network of the Conclave
Giuseppe Soda, Alessandro Iorio, Leonardo Rizzo (2025)

https://arxiv.org/abs/2505.17635

A cleaner and more detailed version of this file will be uploaded on https://github.com/leonardorizzo/Conclave2025/ in the following days

In [2]:
import networkx as nx
import pandas as pd
from collections import defaultdict

In [3]:
# Create Consecration Network

G_consecration = nx.from_pandas_adjacency(pd.read_excel('data/adjacency_matrix_consecrator.xlsx', index_col=0), create_using=nx.Graph())

In [4]:
#Create Formal Network (person x person projection)

df1=pd.read_excel('data/formal.xlsx')
all_cardinals = set(df1['Person'].unique())

G_formal = nx.Graph()
G_formal.add_nodes_from(all_cardinals)

weights = defaultdict(int)

# Process Group co-affiliations from df1
groups = df1.groupby('Membership')['Person'].apply(list).to_dict()
for group, cardinals in groups.items():
    if len(cardinals) > 1:  # Only consider groups with at least 2 cardinals
        for i in range(len(cardinals)):
            for j in range(i+1, len(cardinals)):
                cardinal_pair = tuple(sorted([cardinals[i], cardinals[j]]))
                weights[cardinal_pair] += 1
for (cardinal1, cardinal2), weight in weights.items():
    G_formal.add_edge(cardinal1, cardinal2, weight=weight)

In [5]:
def add_networks(G1, G2, weight_attr='weight', 
                default_weight=1.0, combine_node_attrs=True):
    """
    Add two weighted networks by combining their edges and summing weights.
    """
    if type(G1) != type(G2):
        raise ValueError("Both graphs must be the same type (directed or undirected)")
    result = type(G1)()
    # Add all nodes from both graphs
    result.add_nodes_from(G1.nodes())
    result.add_nodes_from(G2.nodes())
    
    # If combine_node_attrs is True, merge node attributes
    if combine_node_attrs:
        # First add all G1 node attributes
        for node in G1.nodes():
            result.nodes[node].update(G1.nodes[node])
        # Then add G2 node attributes (overriding G1 if necessary)
        for node in G2.nodes():
            result.nodes[node].update(G2.nodes[node])
    # Add all edges from G1
    for u, v, data in G1.edges(data=True):
        weight = data.get(weight_attr, default_weight)
        result.add_edge(u, v, **{weight_attr: weight})
        # Copy other edge attributes
        edge_data = {k: v for k, v in data.items() if k != weight_attr}
        if edge_data:
            result[u][v].update(edge_data)
    # Add or update edges from G2
    for u, v, data in G2.edges(data=True):
        weight2 = data.get(weight_attr, default_weight)
        
        if result.has_edge(u, v):
            # Edge already exists - add weights
            current_weight = result[u][v].get(weight_attr, default_weight)
            result[u][v][weight_attr] = current_weight + weight2
            
            # Merge other edge attributes (G2 attributes override G1)
            edge_data = {k: v for k, v in data.items() if k != weight_attr}
            if edge_data:
                result[u][v].update(edge_data)
        else:
            # New edge from G2
            result.add_edge(u, v, **{weight_attr: weight2})
            # Copy other edge attributes
            edge_data = {k: v for k, v in data.items() if k != weight_attr}
            if edge_data:
                result[u][v].update(edge_data)
    return result

In [6]:
#Create multiplex network by combining consecration and formal networks
G_multiplex = add_networks(G_consecration, G_formal, weight_attr='weight', combine_node_attrs=True)

In [7]:
# Add attributes to the multiplex network
def load_node_attributes(G, excel_file, node_col='Surname'):
    """
    Load node attributes from Excel and attach to NetworkX graph.
    """
    # Read node attributes from Excel
    node_df = pd.read_excel(excel_file)
    # Set node column as index
    node_df = node_df.set_index(node_col)
    # Convert to dictionary format for NetworkX
    node_attrs = node_df.to_dict('index')
    # Attach attributes to graph
    nx.set_node_attributes(G, node_attrs)
    
    return G

G_multiplex = load_node_attributes(G_multiplex, 'data/node_info.xlsx')

In [8]:
def compute_eigenvector_centrality(G):
    """
    Compute eigenvector centrality for all nodes in graph G,
    order them from highest to lowest, and return the node with highest centrality.
    """
    # Compute eigenvector centrality
    try:
        centrality = nx.eigenvector_centrality(G,weight='weight')
    except nx.PowerIterationFailedConvergence:
        # If the standard algorithm doesn't converge, try with more iterations
        centrality = nx.eigenvector_centrality_numpy(G)
    
    # Order nodes by centrality (highest to lowest)
    ordered_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)
    # Node with highest centrality is the first in the ordered list
    highest_node = ordered_nodes[0][0]
    return highest_node, ordered_nodes


highest_node, ordered_nodes = compute_eigenvector_centrality(G_multiplex)

print(f"Node with highest eigenvector centrality: {highest_node}")
print("\nAll nodes ordered by eigenvector centrality (highest to lowest):")
eigen_centr=[]
for node, centrality in ordered_nodes:
    print(f"Node {node}: {centrality:.6f}")or centrality
    eigen_centr.append(node)

Node with highest eigenvector centrality: Prevost

All nodes ordered by eigenvector centrality (highest to lowest):
Node Prevost: 0.240016
Node Tagle: 0.232220
Node Gugerotti: 0.231541
Node Fernández: 0.214011
Node Parolin: 0.200180
Node Mendonça: 0.196452
Node You Heung-Sik: 0.196135
Node Roche: 0.171134
Node Farrell: 0.167118
Node Turkson: 0.155580
Node Sarah: 0.154604
Node Mamberti: 0.146575
Node Sturla Berhouet: 0.144329
Node Koch: 0.133298
Node Ghirlanda: 0.125052
Node Dew: 0.122899
Node Schönborn: 0.120780
Node Ouédraogo: 0.119513
Node Tempesta: 0.115283
Node Scherer: 0.114875
Node Tobin: 0.114532
Node Lacroix: 0.113832
Node Semeraro: 0.113463
Node Tscherrig: 0.112885
Node Erdő: 0.109792
Node Czerny: 0.108145
Node Zuppi: 0.107030
Node Arborelius: 0.106662
Node Rugambwa: 0.106470
Node Patabendige Don: 0.104648
Node Bozanić: 0.104648
Node Furtado: 0.104648
Node Ladaria Ferrer: 0.102965
Node Grech: 0.099552
Node Nichols: 0.099515
Node Dolan: 0.094571
Node Harvey: 0.093014
Node Filon

In [9]:
def compute_betweenness_centrality(G):
    """
    Compute betweenness centrality for all nodes
    """
    centrality = nx.betweenness_centrality(G,weight='weight')
    # Order nodes by centrality (highest to lowest)
    ordered_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)
    # Node with highest centrality is the first in the ordered list
    highest_node = ordered_nodes[0][0]
    
    return highest_node, ordered_nodes


highest_node_b, ordered_nodes_b = compute_betweenness_centrality(G_multiplex)

print(f"Node with highest eigenvector centrality: {highest_node}")
print("\nAll nodes ordered by eigenvector centrality (highest to lowest):")
betw_centr=[]
for node, centrality in ordered_nodes_b:
    print(f"Node {node}: {centrality:.6f}")or centrality 
    betw_centr.append(node)

Node with highest eigenvector centrality: Prevost

All nodes ordered by eigenvector centrality (highest to lowest):
Node Re: 0.056985
Node Bertone: 0.056165
Node Betori: 0.034384
Node Tobin: 0.030965
Node Romeo: 0.030908
Node Zuppi: 0.030816
Node Czerny: 0.030247
Node De Donatis: 0.029256
Node Tagle: 0.027344
Node Semeraro: 0.026821
Node Calcagno: 0.026659
Node Erdő: 0.026649
Node Koch: 0.024770
Node Burke: 0.024515
Node Prevost: 0.024025
Node Fernández: 0.023702
Node Parolin: 0.023121
Node Farrell: 0.021393
Node Chow Sau-Yan: 0.020830
Node Cobo Cano: 0.020744
Node Filoni: 0.020681
Node Errázuriz Ossa: 0.020677
Node Barreto Jimeno: 0.020604
Node Osoro Sierra: 0.020344
Node Mangkhanekhoun: 0.019153
Node Gambetti: 0.018797
Node Tscherrig: 0.017353
Node Gugerotti: 0.017317
Node Mendonça: 0.016216
Node Krajewski: 0.015671
Node Cantoni: 0.015140
Node Mcelroy: 0.014761
Node Harvey: 0.014289
Node Sturla Berhouet: 0.014195
Node Suharyo Hardjoatmodjo: 0.013770
Node Baldisseri: 0.013717
Node Ryś

In [10]:
def coalition_formation_index(G):
    """
    Calculate a coalition formation index for each node based on:
    1. Community density (density of connections within node's community)
    2. Ego network size (degree of the node)
    3. Reach (proportion of other communities that the node connects to)
    
    Higher values indicate better coalition formation capability
    """
    # Detect communities using modularity-based method
    communities = list(nx.community.greedy_modularity_communities(G))
    print(len(communities))
    n_communities = len(communities)
    print(f"Number of communities detected: {n_communities}")
    
    # Map nodes to their communities
    community_map = {}
    for i, comm in enumerate(communities):
        for node in comm:
            community_map[node] = i
    
    # Calculate density within each community
    community_density = {}
    for i, comm in enumerate(communities):
        comm_subgraph = G.subgraph(comm)
        n_nodes = len(comm)
        n_edges = comm_subgraph.number_of_edges()
        max_possible_edges = (n_nodes * (n_nodes - 1)) / 2
        # Avoid division by zero for single-node communities
        if max_possible_edges > 0:
            density = n_edges / max_possible_edges
        else:
            density = 0
        community_density[i] = density
    
    # Calculate reach (share of other communities connected to) for each node
    reach = {}
    for node in G.nodes():
        node_comm = community_map[node]
        connected_communities = set()
        # Find all communities this node connects to
        for neighbor in G.neighbors(node):
            neighbor_comm = community_map[neighbor]
            if neighbor_comm != node_comm:
                connected_communities.add(neighbor_comm)
        # Calculate share of other communities reached
        # Handle edge case where there's only one community
        if n_communities <= 1:
            reach[node] = 0
        else:
            reach[node] = len(connected_communities) / (n_communities - 1)
    
    # Calculate the coalition formation index
    cfi = {}
    for node in G.nodes():
        # Get community density for this node's community
        node_comm = community_map[node]
        density_factor = community_density[node_comm]
        # Get ego network size (degree)
        degree_factor = G.degree(node)
        # Get reach across communities (already normalized between 0-1)
        reach_factor = reach[node]
        # Combine factors into a single score
        # Weight parameters based on importance
        density_weight = 0.15
        degree_weight = 0.15
        reach_weight = 0.7
        # Normalize degree factor only (reach is already normalized, density is 0-1)
        max_degree = max(dict(G.degree()).values())
        normalized_degree = degree_factor / max_degree if max_degree > 0 else 0
        
        # Calculate final score
        cfi[node] = (density_factor * density_weight + 
                      normalized_degree * degree_weight + 
                      reach_factor * reach_weight)
    
    return cfi, community_density, reach, community_map

cfi, community_density, reach, community_map = coalition_formation_index(G_multiplex)
sorted_cfi = sorted(cfi.items(), key=lambda x: x[1], reverse=True)
sorted_cfi

12
Number of communities detected: 12


[('Tagle', 0.4440626751801364),
 ('Czerny', 0.3961635479440887),
 ('Parolin', 0.37381158797744984),
 ('Mamberti', 0.3449125054086425),
 ('Burke', 0.3424938231734465),
 ('Schönborn', 0.34078406504167),
 ('Koch', 0.33333750347511815),
 ('Dolan', 0.3325271843077251),
 ('Prevost', 0.3305514157973174),
 ('Filoni', 0.3281327335621214),
 ('Erdő', 0.326456769530164),
 ('Bertone', 0.31738956962882603),
 ('Ouellet', 0.31299511888322235),
 ('Baldisseri', 0.30913268889488105),
 ('Calcagno', 0.3077565421058902),
 ('Mendonça', 0.3060467839741138),
 ('Krajewski', 0.30362810173891774),
 ('Fernández', 0.3027286627745343),
 ('You Heung-Sik', 0.3016523332285101),
 ('Osoro Sierra', 0.3006097977823049),
 ('Petrocchi', 0.3006097977823049),
 ('Gugerotti', 0.2999763691965527),
 ('Sarah', 0.29366146287319633),
 ('Farrell', 0.2933954524945651),
 ('Betori', 0.29097677025936913),
 ('Roche', 0.2909091692952147),
 ('Sturla Berhouet', 0.2906431589165835),
 ('Abril Y Castelló', 0.28415346121768137),
 ('Tscherrig', 0.